In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
#reading in the data

routes = pd.read_csv('routes.txt')

trips = pd.read_csv('trips.txt')

stops = pd.read_csv('stop_times.txt', low_memory=False)

calendar = pd.read_csv('calendar.txt')
exceptions = pd.read_csv('calendar_dates.txt')


In [3]:
# reducing stops table to necessary data only

stops.columns

stops = stops[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence', 'shape_dist_traveled','timepoint']]

# Select only the rows with the minimum and maximum 'stop_sequence' for each unique 'trip_id'
min_max_stops = stops.loc[stops.groupby('trip_id')['stop_sequence'].idxmin()]
min_max_stops = min_max_stops.append(stops.loc[stops.groupby('trip_id')['stop_sequence'].idxmax()])
min_max_stops

C:\Users\oxant\AppData\Local\Temp\ipykernel_9448\3676927779.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  min_max_stops = min_max_stops.append(stops.loc[stops.groupby('trip_id')['stop_sequence'].idxmax()])


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,timepoint
44777,1,10:10:00,10:10:00,14004939,1,0.000000,1
44705,2,06:58:00,06:58:00,14004939,1,0.000000,1
44729,3,07:40:00,07:40:00,14004939,1,0.000000,1
44506,4,19:40:00,19:40:00,14004939,1,0.000000,1
44482,5,18:45:00,18:45:00,14004939,1,0.000000,1
...,...,...,...,...,...,...,...
110765,21027,13:34:00,13:34:00,14008083,35,13732.725174,1
110835,21028,14:49:00,14:49:00,14008083,35,13732.725174,1
110555,21029,09:39:00,09:39:00,14008083,35,13732.725174,1
110625,21030,10:54:00,10:54:00,14008083,35,13732.725174,1


In [4]:
stops.loc[stops.groupby('trip_id')['stop_sequence'].idxmin()]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,timepoint
44777,1,10:10:00,10:10:00,14004939,1,0.0,1
44705,2,06:58:00,06:58:00,14004939,1,0.0,1
44729,3,07:40:00,07:40:00,14004939,1,0.0,1
44506,4,19:40:00,19:40:00,14004939,1,0.0,1
44482,5,18:45:00,18:45:00,14004939,1,0.0,1
...,...,...,...,...,...,...,...
110731,21027,12:55:00,12:55:00,14008004,1,0.0,1
110801,21028,14:10:00,14:10:00,14008004,1,0.0,1
110521,21029,09:00:00,09:00:00,14008004,1,0.0,1
110591,21030,10:15:00,10:15:00,14008004,1,0.0,1


In [5]:
# Calculate the time difference between the first and last stop sequence for each trip_id and convert to hours
min_max_stops['trip_duration'] = min_max_stops.groupby('trip_id')['departure_time'].transform(lambda x: (pd.to_datetime(x).max() - pd.to_datetime(x).min()).total_seconds() / 3600)

min_max_stops

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,timepoint,trip_duration
44777,1,10:10:00,10:10:00,14004939,1,0.000000,1,0.650000
44705,2,06:58:00,06:58:00,14004939,1,0.000000,1,0.700000
44729,3,07:40:00,07:40:00,14004939,1,0.000000,1,0.633333
44506,4,19:40:00,19:40:00,14004939,1,0.000000,1,0.733333
44482,5,18:45:00,18:45:00,14004939,1,0.000000,1,0.666667
...,...,...,...,...,...,...,...,...
110765,21027,13:34:00,13:34:00,14008083,35,13732.725174,1,0.650000
110835,21028,14:49:00,14:49:00,14008083,35,13732.725174,1,0.650000
110555,21029,09:39:00,09:39:00,14008083,35,13732.725174,1,0.650000
110625,21030,10:54:00,10:54:00,14008083,35,13732.725174,1,0.650000


In [6]:
# calculate bus speed

min_max_stops['speed'] = min_max_stops['shape_dist_traveled']/(1000*min_max_stops['trip_duration'])

In [7]:
# Now select only entries for the last stop, remove entries for the 1st stop of the trip
min_max_stops = min_max_stops[min_max_stops['stop_sequence'] != 1]
min_max_stops.describe()

,trip_id,stop_id,stop_sequence,shape_dist_traveled,timepoint,trip_duration,speed
count,7067.000000,7.067000e+03,7067.000000,7067.000000,7067.000000,7067.000000,7067.000000
mean,3593.278336,1.400436e+07,37.516061,19987.086145,0.998868,0.705931,26.603834
std,2324.656770,2.778971e+03,15.613480,18288.929848,0.033629,0.407792,6.969872
min,1.000000,1.400106e+07,4.000000,3808.110805,0.000000,0.100000,13.987626
25%,1767.500000,1.400253e+07,28.000000,10406.747951,1.000000,0.416667,22.627042
50%,3534.000000,1.400253e+07,34.000000,12975.730830,1.000000,0.533333,25.352968
75%,5300.500000,1.400799e+07,42.000000,25882.780541,1.000000,0.833333,28.080060
max,21031.000000,1.400897e+07,77.000000,135294.119197,1.000000,2.566667,65.767724


In [8]:
min_max_stops.loc[min_max_stops['speed'].idxmax()]

trip_id                        6323
arrival_time               14:40:00
departure_time             14:40:00
stop_id                    14002221
stop_sequence                    13
shape_dist_traveled    43845.149098
timepoint                         1
trip_duration              0.666667
speed                     65.767724
Name: 1258, dtype: object

In [9]:
routes.describe()

,agency_id,route_id,route_desc,route_type,route_url,route_sort_order,min_headway_minutes,eligibility_restricted,continuous_pickup,continuous_drop_off,tts_route_short_name,tts_route_long_name
count,38.0,38.000000,0.0,38.000000,0.0,38.000000,3.000000,38.0,38.0,38.0,0.0,0.0
mean,558.0,8549.131579,NaN,2.973684,NaN,30.315789,70.000000,0.0,1.0,1.0,NaN,NaN
std,0.0,10158.345578,NaN,0.162221,NaN,23.013633,45.825757,0.0,0.0,0.0,NaN,NaN
min,558.0,3771.000000,NaN,2.000000,NaN,0.000000,30.000000,0.0,1.0,1.0,NaN,NaN
25%,558.0,4620.500000,NaN,3.000000,NaN,9.250000,45.000000,0.0,1.0,1.0,NaN,NaN
50%,558.0,4655.000000,NaN,3.000000,NaN,25.000000,60.000000,0.0,1.0,1.0,NaN,NaN
75%,558.0,9032.750000,NaN,3.000000,NaN,51.750000,90.000000,0.0,1.0,1.0,NaN,NaN
max,558.0,65445.000000,NaN,3.000000,NaN,74.000000,120.000000,0.0,1.0,1.0,NaN,NaN


In [10]:
# covert trip_id to integer which is currently as object and therefore is not possible to do a merge
min_max_stops['trip_id'] = min_max_stops['trip_id'].astype(int)

C:\Users\oxant\AppData\Local\Temp\ipykernel_9448\645666959.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  min_max_stops['trip_id'] = min_max_stops['trip_id'].astype(int)


In [11]:
# Merge dataframes based on 'trip_id' column with 'outer' join
stops_trips = pd.merge(trips[['route_id', 'service_id', 'trip_id', 'trip_short_name', 'trip_headsign', 'direction_id',
                             'block_id', 'shape_id', 'operator_id']], min_max_stops, on=['trip_id'], how='outer')

stops_trips.head(20)

,route_id,service_id,trip_id,trip_short_name,trip_headsign,direction_id,block_id,shape_id,operator_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,timepoint,trip_duration,speed
0,11712,WKO00_MTWThF_20211220_20220131_1,6985,NaN,O'Sullivan Dr,1,6985,11712,38,14:50:00,14:50:00,14001676,27,19467.197481,1,0.833333,23.360637
1,11712,WKO00_MTWThF_20211220_20220131_1,6984,NaN,O'Sullivan Dr,1,6984,11712,38,09:50:00,09:50:00,14001676,27,19467.197481,1,0.833333,23.360637
2,11712,WKO00_MTWThF_20211220_20220131_1,6989,NaN,O'Sullivan Dr,1,6989,11712,38,10:50:00,10:50:00,14001676,27,19467.197481,1,0.833333,23.360637
3,11712,WKO00_MTWThF_20211220_20220131_1,6988,NaN,O'Sullivan Dr,1,6988,11712,38,11:50:00,11:50:00,14001676,27,19467.197481,1,0.833333,23.360637
4,11712,WKO00_MTWThF_20211220_20220131_1,6987,NaN,O'Sullivan Dr,1,6987,11712,38,12:50:00,12:50:00,14001676,27,19467.197481,1,0.833333,23.360637
5,11712,WKO00_MTWThF_20211220_20220131_1,6986,NaN,O'Sullivan Dr,1,6986,11712,38,13:50:00,13:50:00,14001676,27,19467.197481,1,0.833333,23.360637
6,11712,WKO00_MTWThF_20211004_20211219_1,6990,NaN,O'Sullivan Dr,1,6990,11712,38,13:50:00,13:50:00,14001676,27,19467.197481,1,0.833333,23.360637
7,11712,WKO00_MTWThF_20211004_20211219_1,6991,NaN,O'Sullivan Dr,1,6991,11712,38,09:50:00,09:50:00,14001676,27,19467.197481,1,0.833333,23.360637
8,11712,WKO00_MTWThF_20211004_20211219_1,6992,NaN,O'Sullivan Dr,1,6992,11712,38,10:50:00,10:50:00,14001676,27,19467.197481,1,0.833333,23.360637
9,11712,WKO00_MTWThF_20211004_20211219_1,6994,NaN,O'Sullivan Dr,1,6994,11712,38,12:50:00,12:50:00,14001676,27,19467.197481,1,0.833333,23.360637


In [12]:
# merge tirp, stop_times and routes data

routes_trips_stops = pd.merge(routes[['agency_id','route_id','route_short_name','route_long_name','route_type',
                                      'route_sort_order','min_headway_minutes']], stops_trips, on=['route_id'], how='outer')
routes_trips_stops.describe()

,agency_id,route_id,route_type,route_sort_order,min_headway_minutes,trip_id,trip_short_name,direction_id,block_id,operator_id,stop_id,stop_sequence,shape_dist_traveled,timepoint,trip_duration,speed
count,7069.0,7069.000000,7069.000000,7069.000000,456.000000,7067.000000,98.000000,7067.000000,7067.000000,7067.000000,7.067000e+03,7067.000000,7067.000000,7067.000000,7067.000000,7067.000000
mean,558.0,7558.733201,2.985571,28.623992,72.631579,3593.278336,104.500000,0.501769,3593.278336,37.955851,1.400436e+07,37.516061,19987.086145,0.998868,0.705931,26.603834
std,0.0,8097.799486,0.119260,22.822519,44.188460,2324.656770,0.502571,0.500032,2324.656770,2.590295,2.778971e+03,15.613480,18288.929848,0.033629,0.407792,6.969872
min,558.0,3771.000000,2.000000,0.000000,30.000000,1.000000,104.000000,0.000000,1.000000,0.000000,1.400106e+07,4.000000,3808.110805,0.000000,0.100000,13.987626
25%,558.0,4608.000000,3.000000,6.000000,30.000000,1767.500000,104.000000,0.000000,1767.500000,38.000000,1.400253e+07,28.000000,10406.747951,1.000000,0.416667,22.627042
50%,558.0,4625.000000,3.000000,21.000000,45.000000,3534.000000,104.500000,1.000000,3534.000000,38.000000,1.400253e+07,34.000000,12975.730830,1.000000,0.533333,25.352968
75%,558.0,9022.000000,3.000000,46.000000,120.000000,5300.500000,105.000000,1.000000,5300.500000,38.000000,1.400799e+07,42.000000,25882.780541,1.000000,0.833333,28.080060
max,558.0,65445.000000,3.000000,74.000000,120.000000,21031.000000,105.000000,1.000000,21031.000000,44.000000,1.400897e+07,77.000000,135294.119197,1.000000,2.566667,65.767724


In [13]:
routes_trips_stops.groupby('route_id')['speed'].agg(['mean', 'std'])

,mean,std
route_id,,
3771,27.095411,3.258900
4602,34.535511,4.033524
4604,21.256547,2.419853
4606,25.705126,1.479083
4608,24.150532,1.803727
4610,21.821353,2.369459
4612,23.079672,2.199843
4616,24.773899,1.934790
4618,21.982916,2.470370


In [15]:
routes_trips_stops.head()

,agency_id,route_id,route_short_name,route_long_name,route_type,route_sort_order,min_headway_minutes,service_id,trip_id,trip_short_name,...,shape_id,operator_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,timepoint,trip_duration,speed
0,558,19537,M,Meteor,3,0,NaN,WKO00_MTWThF_20220201_20241109,1024.0,NaN,...,19537_0,38.0,17:23:00,17:23:00,14007566.0,46.0,19708.917971,1.0,0.883333,22.311983
1,558,19537,M,Meteor,3,0,NaN,WKO00_MTWThF_20220201_20241109,1018.0,NaN,...,19537_0,38.0,20:19:00,20:19:00,14007566.0,46.0,19708.917971,1.0,0.733333,26.875797
2,558,19537,M,Meteor,3,0,NaN,WKO00_MTWThF_20220201_20241109,1017.0,NaN,...,19537_0,38.0,20:49:00,20:49:00,14007566.0,46.0,19708.917971,1.0,0.733333,26.875797
3,558,19537,M,Meteor,3,0,NaN,WKO00_MTWThF_20220201_20241109,1016.0,NaN,...,19537_0,38.0,21:19:00,21:19:00,14007566.0,46.0,19708.917971,1.0,0.733333,26.875797
4,558,19537,M,Meteor,3,0,NaN,WKO00_MTWThF_20220201_20241109,1023.0,NaN,...,19537_0,38.0,17:58:00,17:58:00,14007566.0,46.0,19708.917971,1.0,0.883333,22.311983


In [18]:
# Manipulate calendar dataframe

# Convert start_date and end_date columns to datetime
calendar['start_date'] = pd.to_datetime(calendar['start_date'], format='%Y%m%d')
calendar['end_date'] = pd.to_datetime(calendar['end_date'], format='%Y%m%d')

# Filter for year 2022
calendar = calendar[calendar['start_date'].dt.year == 2022]

calendar.head()

,service_id,service_name,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,WKO00_MTWThF_20220201_20220414,00_Secondary School Terms (1st Term only) (Wee...,1,1,1,1,1,0,0,2022-02-01,2022-04-14
1,WKO00_MTWThF_20221017_20221220,00_Secondary School Terms (2nd May) (Weekday),1,1,1,1,1,0,0,2022-10-17,2022-12-20
2,WKO00_MTWThF_20220725_20220930,00_Secondary School Terms (2nd May) (Weekday),1,1,1,1,1,0,0,2022-07-25,2022-09-30
3,WKO00_MTWThF_20220502_20220708,00_Secondary School Terms (2nd May) (Weekday),1,1,1,1,1,0,0,2022-05-02,2022-07-08
4,WKO00_SSu_20220201_20220501,00_Year_Round_Until_1st_May (Weekend),0,0,0,0,0,1,1,2022-02-01,2022-05-01
